<a href="https://colab.research.google.com/github/BrendenLatham/Data_Viz/blob/main/Exam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exam 2

# Brenden Latham 245775

# CPSMA 4313

In [238]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy
import re
import statistics as stat
import math

## 1

### a

I will first import the dataframe

In [239]:
df = pd.read_csv('https://raw.githubusercontent.com/nurfnick/Data_Viz/main/Activity_Dataset_V1.csv')
df.head()

,Unnamed: 0,activity_day,workout_type,distance,time,calories,total_steps,avg_speed,avg_cadence,max_cadence,...,max_pace,min_pace,avg_heart_rate,max_heart_rate,min_heart_rate,vo2_max(%),aerobic(%),anaerobic(%),intensive(%),light(%)
0,0,2022-01-01,Freestyle,9.30,77,123,NaN,18.88,168.54,138.30,...,NaN,NaN,112.5,122.0,103,19,28,2,7,50
1,1,2022-01-01,Freestyle,3.44,96,55,NaN,29.65,125.92,292.81,...,NaN,NaN,111.0,122.0,100,42,28,2,29,88
2,2,2022-01-01,Indoor Cycling,6.34,85,33,NaN,17.85,81.93,323.69,...,NaN,NaN,95.0,90.0,100,1,32,0,22,43
3,3,2022-01-01,Walking,7.91,42,82,1571.0,22.10,29.63,180.16,...,28:58,07:58,83.0,85.0,81,3,22,0,24,65
4,4,2022-01-01,Open Water,8.99,36,131,NaN,25.83,64.55,342.89,...,NaN,NaN,138.0,166.0,110,7,0,5,21,88


Now I am going to take a look at the datatypes of each column

In [240]:
df.dtypes

Unnamed: 0          int64
activity_day       object
workout_type       object
distance          float64
time                int64
calories            int64
total_steps       float64
avg_speed         float64
avg_cadence       float64
max_cadence       float64
avg_pace           object
max_pace           object
min_pace           object
avg_heart_rate    float64
max_heart_rate    float64
min_heart_rate      int64
vo2_max(%)          int64
aerobic(%)          int64
anaerobic(%)        int64
intensive(%)        int64
light(%)            int64
dtype: object

It looks to me like "workout_type" should be a string. Also "total_steps" and "max_heart_rate" should be an int unless they are counting partial steps.

### b

Now I will remove the column "unnamed" because it is unnecesary

In [241]:
df = df.drop(columns = ['Unnamed: 0'])

### c

Now to clean up the column names

In [242]:
df = df.rename(columns=lambda x: re.sub("\([^()]*\)", '', x))
df.columns

Index(['activity_day', 'workout_type', 'distance', 'time', 'calories',
       'total_steps', 'avg_speed', 'avg_cadence', 'max_cadence', 'avg_pace',
       'max_pace', 'min_pace', 'avg_heart_rate', 'max_heart_rate',
       'min_heart_rate', 'vo2_max', 'aerobic', 'anaerobic', 'intensive',
       'light'],
      dtype='object')

### d

Now I will donvert the column "activity_day" to date-time format

In [243]:
df['activity_day'] = pd.to_datetime(df.activity_day, format = '%Y-%m-%d')
df.dtypes

activity_day      datetime64[ns]
workout_type              object
distance                 float64
time                       int64
calories                   int64
total_steps              float64
avg_speed                float64
avg_cadence              float64
max_cadence              float64
avg_pace                  object
max_pace                  object
min_pace                  object
avg_heart_rate           float64
max_heart_rate           float64
min_heart_rate             int64
vo2_max                    int64
aerobic                    int64
anaerobic                  int64
intensive                  int64
light                      int64
dtype: object

### e

I will also be looking at the column "total_steps". Here I will replace the NaN's with the column mean trunctated to the whole number and convert the data type to an int. This is done to make future analysis of the column easier. 

In [244]:
StepMean = df['total_steps'].mean()
StepMean = math.trunc(StepMean)
df['total_steps'] = df['total_steps'].fillna(value = int(StepMean))
df.total_steps = df.total_steps.astype('int')
df['total_steps']

0      4081
1      4081
2      4081
3      1571
4      4081
       ... 
995    4081
996    4081
997    4081
998    4081
999    4081
Name: total_steps, Length: 1000, dtype: int64

### f

Following with the theme of converting columns, I will change the "avg_pace" column into a float representing minutes.

Here is how it looks now

In [245]:
df['avg_pace']

0        NaN
1        NaN
2        NaN
3      07:58
4        NaN
       ...  
995      NaN
996      NaN
997      NaN
998      NaN
999      NaN
Name: avg_pace, Length: 1000, dtype: object

Time to change that

First I will get the values in the form of minutes.

In [246]:
df['avg_pace'] = [re.sub(':','.', str(x)) for x in df['avg_pace']]
df.avg_pace = df.avg_pace.astype('float')
df['avg_pace']

0       NaN
1       NaN
2       NaN
3      7.58
4       NaN
       ... 
995     NaN
996     NaN
997     NaN
998     NaN
999     NaN
Name: avg_pace, Length: 1000, dtype: float64

Now I will raplace all the NaNs with the mean, once again for future data processing reasons.

In [247]:
df['avg_pace'] = df['avg_pace'].fillna(value = df['avg_pace'].mean())
df['avg_pace']

0      7.136453
1      7.136453
2      7.136453
3      7.580000
4      7.136453
         ...   
995    7.136453
996    7.136453
997    7.136453
998    7.136453
999    7.136453
Name: avg_pace, Length: 1000, dtype: float64

This still isn't right yet as I just replaced the comma with a decimal place. Time to do some math

In [248]:
for i in range(0,len(df['avg_pace']),1):
  dec = df['avg_pace'][i]%1
  dec = dec/.6
  df['avg_pace'][i] = trunc(df['avg_pace'][i])+dec
df['avg_pace']

0      7.227422
1      7.227422
2      7.227422
3      7.966667
4      7.227422
         ...   
995    7.227422
996    7.227422
997    7.227422
998    7.227422
999    7.227422
Name: avg_pace, Length: 1000, dtype: float64

And there we have it

### g

Now I will gather some data on calories per workout type

In [249]:
df.groupby(by='workout_type').agg(['mean','median','count','std'])['calories']

,mean,median,count,std
workout_type,,,,
Cricket,307.548387,330.0,93,149.950609
Freestyle,278.552083,294.0,96,163.703768
Indoor Cycling,280.450000,306.0,80,156.633322
Open Water,296.747253,328.0,91,160.068499
Outdoor Cycling,299.129412,301.0,85,158.731263
Outdoor Running,301.469136,349.0,81,165.725080
Pool Swimming,283.414894,300.0,94,157.576703
Trail Run,267.966667,264.0,90,155.748533
Treadmill,278.142857,269.5,98,146.963352


Note you could get this for all columns by deleting the last brackets, or get this for any other column by changing the contents of the brackets.

### h

Next I will create an indicator column indicating grater than or equal to 30% aerobic activity

First however I will be reformatting the columns that are in percent because the current format is getting annoying.

In [250]:
cols = ('vo2_max','aerobic','anaerobic','intensive','light')
for col in cols:
  df[col] = df[col].astype('float')
  for j in range(0,len(df[col]),1):
    df[col][j] = df[col][j]/100
df.head()

,activity_day,workout_type,distance,time,calories,total_steps,avg_speed,avg_cadence,max_cadence,avg_pace,max_pace,min_pace,avg_heart_rate,max_heart_rate,min_heart_rate,vo2_max,aerobic,anaerobic,intensive,light
0,2022-01-01,Freestyle,9.30,77,123,4081,18.88,168.54,138.30,7.227422,NaN,NaN,112.5,122.0,103,0.19,0.28,0.02,0.07,0.50
1,2022-01-01,Freestyle,3.44,96,55,4081,29.65,125.92,292.81,7.227422,NaN,NaN,111.0,122.0,100,0.42,0.28,0.02,0.29,0.88
2,2022-01-01,Indoor Cycling,6.34,85,33,4081,17.85,81.93,323.69,7.227422,NaN,NaN,95.0,90.0,100,0.01,0.32,0.00,0.22,0.43
3,2022-01-01,Walking,7.91,42,82,1571,22.10,29.63,180.16,7.966667,28:58,07:58,83.0,85.0,81,0.03,0.22,0.00,0.24,0.65
4,2022-01-01,Open Water,8.99,36,131,4081,25.83,64.55,342.89,7.227422,NaN,NaN,138.0,166.0,110,0.07,0.00,0.05,0.21,0.88


I will now create the indicator column

In [251]:
df['aerobic_indicator'] = 0
for k in range(0,len(df['aerobic']),1):
  if df['aerobic'][k] >= .3:
   df['aerobic_indicator'][k] = 1
  else:
    df['aerobic_indicator'][k] = 0
df.head()

,activity_day,workout_type,distance,time,calories,total_steps,avg_speed,avg_cadence,max_cadence,avg_pace,...,min_pace,avg_heart_rate,max_heart_rate,min_heart_rate,vo2_max,aerobic,anaerobic,intensive,light,aerobic_indicator
0,2022-01-01,Freestyle,9.30,77,123,4081,18.88,168.54,138.30,7.227422,...,NaN,112.5,122.0,103,0.19,0.28,0.02,0.07,0.50,0
1,2022-01-01,Freestyle,3.44,96,55,4081,29.65,125.92,292.81,7.227422,...,NaN,111.0,122.0,100,0.42,0.28,0.02,0.29,0.88,0
2,2022-01-01,Indoor Cycling,6.34,85,33,4081,17.85,81.93,323.69,7.227422,...,NaN,95.0,90.0,100,0.01,0.32,0.00,0.22,0.43,1
3,2022-01-01,Walking,7.91,42,82,1571,22.10,29.63,180.16,7.966667,...,07:58,83.0,85.0,81,0.03,0.22,0.00,0.24,0.65,0
4,2022-01-01,Open Water,8.99,36,131,4081,25.83,64.55,342.89,7.227422,...,NaN,138.0,166.0,110,0.07,0.00,0.05,0.21,0.88,0


### i

Finally, I will show which day of the week and workout type has the maximum 'max_cadence'

first the day

I will start by adding a column with the day of the week

In [255]:
df['weekday'] = 0
days = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']
for l in range(0,len(df['activity_day']),1):
  df['weekday'][l] = days[df['activity_day'][l].weekday()]
df['weekday']

0      saturday
1      saturday
2      saturday
3      saturday
4      saturday
         ...   
995      monday
996      monday
997      monday
998      monday
999      monday
Name: weekday, Length: 1000, dtype: object

My code above might be a little confusing, but let me explain. I created a new column for day of the week and initialized all values to 0. I will be using the .weekday() function from datetime which returns a value in the range of 0-6 corresponding to monday-sunday respectively. Before my loop I created a list with the value in each index in that list corresponding to that day. IE: day[0] = monday, day[1] = tuesday ... I then set each row of the new column to be equal to the value contained in the index number that .weekday() returned, which corresponded with that day as a string

Now moving on the finding the maxes

In [262]:
df.groupby(by='weekday').agg(['max'])['max_cadence']

,max
weekday,
friday,347.81
monday,348.96
saturday,349.76
sunday,346.43
thursday,347.40
tuesday,349.39
wednesday,349.78


It looks like saturday had the max 'max_cadence'

Now for grouped by workout type

In [259]:
df.groupby(by='workout_type').agg(['max'])['max_cadence']

,max
workout_type,
Cricket,331.36
Freestyle,347.81
Indoor Cycling,342.94
Open Water,349.76
Outdoor Cycling,346.40
Outdoor Running,348.80
Pool Swimming,349.78
Trail Run,341.55
Treadmill,349.66


And the max here was pool swimming

Now I will do which pair of the two had the max

In [264]:
df.groupby(by=['weekday','workout_type']).agg(['max'])['max_cadence']

max
weekday   workout_type           
friday    Cricket          217.25
          Freestyle        347.81
          Indoor Cycling   290.15
          Open Water       319.61
          Outdoor Cycling  282.35
...                           ...
wednesday Pool Swimming    349.78
          Trail Run        286.10
          Treadmill        349.66
          Trekking         339.65
          Walking          337.74

[77 rows x 1 columns]

In [281]:
WorkoutDay = df.groupby(by=['weekday','workout_type']).agg(['max'])['max_cadence']
WorkoutDay = WorkoutDay[WorkoutDay == WorkoutDay.max()].dropna()
WorkoutDay

,,max
weekday,workout_type,
wednesday,Pool Swimming,349.78


wednesday pool swimming had the max 'max_cadence'

## 2

### a

2.5 hours

### b

Hardest was regex. Easiest was types

### c

It seems like regex could have it's own whole 8 week course but I am very glad we didn't do that.

Sources:

https://www.adamsmith.haus/python/answers/how-to-use-regular-expressions-to-remove-text-within-parentheses-in-python#:~:text=Use%20re.,to%20remove%20text%20within%20parentheses&text=sub

https://kodify.net/python/math/truncate-integers/

https://stackoverflow.com/questions/48447123/convert-time-hhmmss-to-minutes-in-python

https://stackoverflow.com/questions/50554107/typeerror-unrecognized-value-type-class-str

https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

https://stackoverflow.com/questions/9847213/how-do-i-get-the-day-of-week-given-a-date

https://github.com/BrendenLatham/Data_Viz